# The Battle of Neighborhoods (Week 2)

## Table of contents
1. Introduction & Business Problem
2. Data

## Introduction & Business Problem 
In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an restaurant in Toronto, Canada.

Here we will try finding if someone wants to open a new restaurant in the city which location is best suited for it keeping in mind the competitors and which income group of people will be attracted most to it based on the population of the neighborhood.

Since there are lots of restaurants in Toronto we will try to detect locations that are not already crowded with restaurants. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data 
Based on definition of our problem, factors that will influence our decision are,

 - All existing restaurants in the neighborhood (any type of restaurant)
 - Age group of people with their income
 - Distance of neighborhood from city center
We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

> Following data sources will be needed to extract/generate the required information,

 - Centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
 - Number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
column_names = ['Postal_Code','Borough', 'Neighborhood'] # define the dataframe columns
df = pd.DataFrame(columns=column_names)

**Download the Data & Explore**

In [2]:
!conda install -c conda beautifulsoup4 --yes
!conda install -c conda lxml --yes
print("Success!!")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.2       |           py36_0         161 KB
    certifi-2019.11.28         |           py36_0         156 KB
    soupsieve-1.9.5            |           py36_0          61 KB
    openssl-1.1.1d             |       h7b6447c_4         3.7 MB
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.5-py36_0                 

The following packages will b

In [3]:
from urllib.request import urlopen
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urlopen(wiki)

from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XkUwXgpAMFkAAG1vKFYAAADB","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":935851093,"wgRevisionId":935851093,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

In [4]:
torontotable=soup.find('table', class_='wikitable sortable')
torontotable

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [5]:
#Generate lists
Postal=[]
Borough=[]
Neighbor=[]

for row in torontotable.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: 
        Postal.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbor.append(cells[2].find(text=True))
        
#Add Data to our DataFrame
df['Postal_Code']=Postal
df['Borough']=Borough
df['Neighborhood']=Neighbor
df

,Postal_Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned\n
9,M9A,Queen's Park,Not assigned\n


## Data Cleaning

In [6]:
df = df.drop(df[df['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)
df.index = pd.RangeIndex(len(df.index))
df

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned\n
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North\n


In [7]:
column_n = ['NEBRVAL'] 
NEBR_NA = pd.DataFrame(columns=column_n)
NA=['Not assigned'] 
NEBR_NA['NEBRVAL'] = NA
df_na=df
for row_index,row in df.iterrows():
    if((df.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (df.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
       
        df_na.loc[row_index,['Neighborhood']] = df.loc[row_index,['Borough']].values.astype('str') 
df_na

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North\n


In [8]:
df_cleanse=df_na.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()

column_names = ['Postal_Code','Borough', 'Neighborhood'] 
df_cleanse = pd.DataFrame(columns=column_names)

df_cleanse = df_na.drop(df_na[df_na['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
df_cleanse.index = pd.RangeIndex(len(df_cleanse.index))
df_cleanse

,Postal_Code,Borough,Neighborhood
0,M5A,Downtown Toronto,Harbourfront
1,M7A,Downtown Toronto,Queen's Park
2,M5B,Downtown Toronto,Ryerson\n
3,M5B,Downtown Toronto,Garden District\n
4,M5C,Downtown Toronto,St. James Town
5,M4E,East Toronto,The Beaches
6,M5E,Downtown Toronto,Berczy Park
7,M5G,Downtown Toronto,Central Bay Street\n
8,M6G,Downtown Toronto,Christie\n
9,M5H,Downtown Toronto,Adelaide\n


In [9]:
column = ['Postal_Code','Borough', 'Neighborhood'] 
df_ungrp = pd.DataFrame(columns=column_names)
df_ungrp = df_na.drop(df_na[df_na['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)
df_ungrp.index = pd.RangeIndex(len(df_ungrp.index))
df_ungrp

,Postal_Code,Borough,Neighborhood
0,M5A,Downtown Toronto,Harbourfront
1,M7A,Downtown Toronto,Queen's Park
2,M5B,Downtown Toronto,Ryerson\n
3,M5B,Downtown Toronto,Garden District\n
4,M5C,Downtown Toronto,St. James Town
5,M4E,East Toronto,The Beaches
6,M5E,Downtown Toronto,Berczy Park
7,M5G,Downtown Toronto,Central Bay Street\n
8,M6G,Downtown Toronto,Christie\n
9,M5H,Downtown Toronto,Adelaide\n


In [10]:
!conda install -c conda-forge geopy --yes 
import time
from geopy.geocoders import Nominatim
from geopy.util import get_version
get_version()

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    sci

'1.21.0'

In [11]:
geolocator = Nominatim(scheme='http', user_agent="TorontoExplorer")

for row_index, item in df_ungrp.iterrows():
    
    listungrp = df_ungrp.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = 'Toronto, Ontario, Canada'
    listungrp.astype('str')
    listungrp = np.append(listungrp, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(listungrp , limit = 15)
    time.sleep(5)
    if(location is not None):
        df_ungrp.loc[df_ungrp.index[row_index], 'Latitude'] = location.latitude
        df_ungrp.loc[df_ungrp.index[row_index], 'Longitude'] = location.longitude

In [12]:
print(df_ungrp)

   Postal_Code           Borough                                       Neighborhood   Latitude  Longitude
0          M5A  Downtown Toronto                                       Harbourfront  43.640080 -79.380150
1          M7A  Downtown Toronto                                       Queen's Park  43.659659 -79.390340
2          M5B  Downtown Toronto                                          Ryerson\n  43.658469 -79.378993
3          M5B  Downtown Toronto                                  Garden District\n  43.656500 -79.377114
4          M5C  Downtown Toronto                                     St. James Town  43.669403 -79.372704
5          M4E      East Toronto                                        The Beaches  43.671024 -79.296712
6          M5E  Downtown Toronto                                        Berczy Park  43.647984 -79.375396
7          M5G  Downtown Toronto                               Central Bay Street\n        NaN        NaN
8          M6G  Downtown Toronto              

In [13]:
import json 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
!conda install -c conda-forge folium=0.5.0 --yes  # Importing to use the Foursquare API lab
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [14]:
print('We have {} boroughs and {} neighborhoods.'.format(
        len(df_ungrp['Borough'].unique()),
        df_ungrp.shape[0]
    )
)

df_ungrp.dropna(inplace =True)
df_ungrp.index = pd.RangeIndex(len(df_ungrp.index))

address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="TorontoExplorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

We have 4 boroughs and 74 neighborhoods.
The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
df_ungrp

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
1,M7A,Downtown Toronto,Queen's Park,43.659659,-79.390340
2,M5B,Downtown Toronto,Ryerson\n,43.658469,-79.378993
3,M5B,Downtown Toronto,Garden District\n,43.656500,-79.377114
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
5,M4E,East Toronto,The Beaches,43.671024,-79.296712
6,M5E,Downtown Toronto,Berczy Park,43.647984,-79.375396
7,M6G,Downtown Toronto,Christie\n,43.664111,-79.418405
8,M5H,Downtown Toronto,Adelaide\n,43.650486,-79.379498
9,M5H,Downtown Toronto,King\n,43.648949,-79.377754


In [16]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_ungrp['Latitude'], df_ungrp['Longitude'], df_ungrp['Borough'], df_ungrp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
CLIENT_ID = '2MAJMIXWJ0S2WP5UROZ3KAIVV4A3SC2EFXTP0AMYO5XF1OHA' # your Foursquare ID
CLIENT_SECRET = 'YDK5XA24PGSJ2UGH2QGX3CX4SODMC4DN3CYH4WRTTQOC4U3K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Successfully Logged-In')

Successfully Logged-In


In [18]:
df_ungrp.loc[0]
neighborhood_latitude = np.float(df_ungrp.loc[0,['Latitude']].values)
neighborhood_longitude =  np.float(df_ungrp.loc[0,['Longitude']].values)

**Now lets get the top 100 venues in Harbor Square Park within a radius of 500meters**

In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e46eb1047e0d600263845f1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Harbourfront',
  'headerFullLocation': 'Harbourfront, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 125,
  'suggestedBounds': {'ne': {'lat': 43.6445801045, 'lng': -79.37394296546611},
   'sw': {'lat': 43.635580095499996, 'lng': -79.38635603453389}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e49413e81dc766f3e3d6312',
       'name': 'Harbour Square Park',
       'location': {'address': '25 Queens Quay West',
        'lat': 43.63925269130776,
        'lng': -79.37839547902978,
        'labeledLatLngs': [{'label': 'display',
          'lat

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']    
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Harbour Square Park,Park,43.639253,-79.378395
1,Lake Ontario,Lake,43.638945,-79.379665
2,Harbourfront,Neighborhood,43.639526,-79.380688
3,Miku,Japanese Restaurant,43.641374,-79.377531
4,BeaverTails,Dessert Shop,43.639736,-79.380068


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
toronto_venues = getNearbyVenues(names=df_ungrp['Neighborhood'],
                                   latitudes=df_ungrp['Latitude'],
                                   longitudes=df_ungrp['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson

Garden District

St. James Town
The Beaches
Berczy Park
Christie

Adelaide

King

Richmond

Dovercourt Village
Dufferin

Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West

Riverdale
Design Exchange
Toronto Dominion Centre
Brockton

Exhibition Place
Parkdale Village
The Beaches West

India Bazaar
Commerce Court
Lawrence Park
Roselawn

Davisville North

Forest Hill North
High Park
The Junction South

The Annex
Yorkville
Parkdale
Roncesvalles
Davisville

Harbord

University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East

Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE

Rathnelly
South Hill
Summerhill West

CN Tower
Bathurst Quay

Harbourfront West

King and Spadina
Rosedale
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(3374, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.64008,-79.38015,Harbour Square Park,43.639253,-79.378395,Park
1,Harbourfront,43.64008,-79.38015,Lake Ontario,43.638945,-79.379665,Lake
2,Harbourfront,43.64008,-79.38015,Harbourfront,43.639526,-79.380688,Neighborhood
3,Harbourfront,43.64008,-79.38015,Miku,43.641374,-79.377531,Japanese Restaurant
4,Harbourfront,43.64008,-79.38015,BeaverTails,43.639736,-79.380068,Dessert Shop


In [27]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide\n,100,100,100,100,100,100
Bathurst Quay\n,25,25,25,25,25,25
Berczy Park,100,100,100,100,100,100
Brockton\n,18,18,18,18,18,18
CN Tower,87,87,87,87,87,87
Cabbagetown,49,49,49,49,49,49
Chinatown,86,86,86,86,86,86
Christie\n,55,55,55,55,55,55
Church and Wellesley,84,84,84,84,84,84


In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 287 uniques categories.


## Analyze each neighborhood

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Udon Restaurant,

In [30]:
toronto_onehot.shape

(3374, 287)

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Udon Restaurant,

In [32]:
toronto_grouped.shape

(62, 287)

In [33]:
Top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(Top_venues))
    print('\n')

----Adelaide
----
                 venue  freq
0          Coffee Shop  0.08
1           Restaurant  0.05
2                 Café  0.05
3            Gastropub  0.04
4  American Restaurant  0.04


----Bathurst Quay
----
             venue  freq
0      Coffee Shop  0.16
1             Café  0.12
2  Harbor / Marina  0.08
3             Park  0.08
4           Garden  0.04


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.06
2   Italian Restaurant  0.04
3                Hotel  0.04
4  Japanese Restaurant  0.04


----Brockton
----
                   venue  freq
0                    Bar  0.17
1  Vietnamese Restaurant  0.11
2                   Park  0.11
3      Korean Restaurant  0.06
4      French Restaurant  0.06


----CN Tower----
                venue  freq
0               Hotel  0.09
1         Coffee Shop  0.06
2         Pizza Place  0.06
3                 Bar  0.05
4  Italian Restaurant  0.05


----Cabbagetown----
                 venue

In [34]:
def return_most_common_venues(row, Top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:Top_venues]

In [35]:
Top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(Top_venues):
    try:
        columns.append('{}{} Popular Venues'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Popular Venues'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], Top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,Adelaide\n,Coffee Shop,Restaurant,Café,Gastropub,American Restaurant,Seafood Restaurant,Gym,Clothing Store,Hotel,Cosmetics Shop
1,Bathurst Quay\n,Coffee Shop,Café,Park,Harbor / Marina,Gym / Fitness Center,Pub,Garden,Sculpture Garden,Ramen Restaurant,Bank
2,Berczy Park,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Gym,Beer Bar,Seafood Restaurant
3,Brockton\n,Bar,Vietnamese Restaurant,Park,Pizza Place,Jazz Club,Korean Restaurant,Café,Coffee Shop,Portuguese Restaurant,French Restaurant
4,CN Tower,Hotel,Pizza Place,Coffee Shop,Gym,Italian Restaurant,Bar,Restaurant,Aquarium,Baseball Stadium,Park
5,Cabbagetown,Restaurant,Café,Coffee Shop,Pizza Place,Japanese Restaurant,Italian Restaurant,Beer Store,Bakery,Indian Restaurant,Gastropub
6,Chinatown,Bar,Café,Dessert Shop,Chinese Restaurant,Cocktail Bar,Coffee Shop,Mexican Restaurant,Art Gallery,Dumpling Restaurant,Vegetarian / Vegan Restaurant
7,Christie\n,Korean Restaurant,Coffee Shop,Japanese Restaurant,Grocery Store,Mexican Restaurant,Karaoke Bar,Cocktail Bar,Indian Restaurant,Dessert Shop,Ice Cream Shop
8,Church and Wellesley,Coffee Shop,Sushi Restaurant,Gay Bar,Japanese Restaurant,Restaurant,Café,Fast Food Restaurant,Hotel,Mediterranean Restaurant,Gastropub
9,Commerce Court,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Seafood Restaurant,Steakhouse,Italian Restaurant,American Restaurant


In [36]:
## Cluster Neighborhoods using Kmeans

In [37]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
#print(toronto_grouped_clustering)
#print(toronto_grouped)
# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_[0:63] 
print(labels)

[3 3 3 2 3 3 2 2 3 3 2 2 3 3 3 2 2 3 4 4 3 2 2 3 3 3 2 2 2 3 3 2 2 0 2 2 3
 2 1 2 2 0 3 3 3 2 3 2 2 0 2 2 2 3 2 3 0 3 2 3 3 3]


In [38]:
toronto_merged = df_ungrp
print(toronto_merged.shape)
labels = np.append(labels,labels[0])
print(labels.shape)
# add clustering labels
toronto_merged['Cluster Labels'] = labels.tolist()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

(63, 5)
(63,)


,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,3,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Pizza Place,Chinese Restaurant,Sports Bar,Sporting Goods Shop,Park
1,M7A,Downtown Toronto,Queen's Park,43.659659,-79.390340,3,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Ice Cream Shop,Chinese Restaurant,French Restaurant,Thai Restaurant
2,M5B,Downtown Toronto,Ryerson\n,43.658469,-79.378993,3,Coffee Shop,Clothing Store,Ramen Restaurant,Japanese Restaurant,Café,Burger Joint,Pizza Place,Lounge,Gastropub,Bakery
3,M5B,Downtown Toronto,Garden District\n,43.656500,-79.377114,2,Coffee Shop,Clothing Store,Fast Food Restaurant,Cosmetics Shop,Restaurant,Hotel,Lingerie Store,Ramen Restaurant,Movie Theater,Tea Room
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704,3,Coffee Shop,Pizza Place,Café,Gym / Fitness Center,Grocery Store,Caribbean Restaurant,Breakfast Spot,Library,Bistro,Restaurant


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster1

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
33,West Toronto,0,Bar,Italian Restaurant,Coffee Shop,Café,Mexican Restaurant,Thai Restaurant,Grocery Store,Arts & Crafts Store,Antique Shop,Nail Salon
41,West Toronto,0,Café,Coffee Shop,Bakery,Pizza Place,Gastropub,Frozen Yogurt Shop,Latin American Restaurant,Bookstore,Liquor Store,Mattress Store
49,Central Toronto,0,Bank,Home Service,Arts & Crafts Store,Park,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Festival,Fast Food Restaurant
56,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Bar,Hotel,Restaurant,Sandwich Place,Beer Bar,French Restaurant,Dessert Shop,Park


## Cluster2

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
38,Central Toronto,1,Sushi Restaurant,Italian Restaurant,Trail,Convenience Store,Pub,Coffee Shop,Flower Shop,General Entertainment,Gastropub,Furniture / Home Store


## Cluster3

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
3,Downtown Toronto,2,Coffee Shop,Clothing Store,Fast Food Restaurant,Cosmetics Shop,Restaurant,Hotel,Lingerie Store,Ramen Restaurant,Movie Theater,Tea Room
6,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Gym,Beer Bar,Seafood Restaurant
7,Downtown Toronto,2,Korean Restaurant,Coffee Shop,Japanese Restaurant,Grocery Store,Mexican Restaurant,Karaoke Bar,Cocktail Bar,Indian Restaurant,Dessert Shop,Ice Cream Shop
10,Downtown Toronto,2,Park,Ice Cream Shop,Beer Store,Women's Store,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
11,West Toronto,2,Café,Bar,Pizza Place,Bus Line,Brazilian Restaurant,Park,Coffee Shop,Restaurant,Falafel Restaurant,Egyptian Restaurant
15,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega,Italian Restaurant,Japanese Restaurant,Salad Place,Steakhouse,Sporting Goods Shop
16,West Toronto,2,Bar,Café,Coffee Shop,Bakery,Cocktail Bar,Restaurant,Korean Restaurant,Jazz Club,Italian Restaurant,Thai Restaurant
21,Downtown Toronto,2,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Deli / Bodega,Gastropub,Bar,Bakery,Seafood Restaurant
22,West Toronto,2,Bar,Vietnamese Restaurant,Park,Pizza Place,Jazz Club,Korean Restaurant,Café,Coffee Shop,Portuguese Restaurant,French Restaurant
26,East Toronto,2,Indian Restaurant,Café,Grocery Store,Halal Restaurant,Shopping Plaza,Gastropub,Bistro,Brewery,Burger Joint,Snack Place


## Cluster4

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Pizza Place,Chinese Restaurant,Sports Bar,Sporting Goods Shop,Park
1,Downtown Toronto,3,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Ice Cream Shop,Chinese Restaurant,French Restaurant,Thai Restaurant
2,Downtown Toronto,3,Coffee Shop,Clothing Store,Ramen Restaurant,Japanese Restaurant,Café,Burger Joint,Pizza Place,Lounge,Gastropub,Bakery
4,Downtown Toronto,3,Coffee Shop,Pizza Place,Café,Gym / Fitness Center,Grocery Store,Caribbean Restaurant,Breakfast Spot,Library,Bistro,Restaurant
5,East Toronto,3,Beach,Breakfast Spot,Pizza Place,Tea Room,Thai Restaurant,Bar,Japanese Restaurant,Bakery,Nail Salon,Coffee Shop
8,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Gastropub,American Restaurant,Seafood Restaurant,Gym,Clothing Store,Hotel,Cosmetics Shop
9,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Gastropub,Seafood Restaurant,Japanese Restaurant,Gym,American Restaurant
12,West Toronto,3,Bar,Bakery,Café,Coffee Shop,Beer Store,Mexican Restaurant,Cocktail Bar,Beer Bar,Sandwich Place,Vietnamese Restaurant
13,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Pizza Place,Chinese Restaurant,Sports Bar,Sporting Goods Shop,Park
14,Downtown Toronto,3,Park,Music Venue,Harbor / Marina,Café,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market


## Cluster5

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
18,East Toronto,4,Coffee Shop,Bus Line,Ice Cream Shop,Skating Rink,Grocery Store,Fried Chicken Joint,Sushi Restaurant,Sandwich Place,Chinese Restaurant,Fish & Chips Shop
19,East Toronto,4,Vietnamese Restaurant,Chinese Restaurant,Bakery,Light Rail Station,Bar,Fast Food Restaurant,Baseball Field,Breakfast Spot,Chocolate Shop,Grocery Store
